In [2]:
import pandas as pd
import numpy as np

## Calculating Best Global Heroes

In [ ]:
class best_Heroes:
    def __init__(self, data):
        self.data = data
        self.hero_stats = {}
        self._populate_hero_stats(mirrored = False)
        self.hero_categories = [
            {"name": "Ana", "category": ["Poke", "Brawl", "Dive"]},
            {"name": "Ashe", "category": ["Poke", "Brawl"]},
            {"name": "Bap", "category": ["Brawl", "Poke"]},
            {"name": "Bastion", "category": ["Poke", "Brawl"]},
            {"name": "Brig", "category": ["Dive"]},
            {"name": "Cass", "category": ["Brawl"]},
            {"name": "DVA", "category": ["Brawl", "Dive"]},
            {"name": "Doom", "category": ["Brawl", "Dive"]},
            {"name": "Echo", "category": ["Dive", "Brawl"]},
            {"name": "Genji", "category": ["Dive", "Brawl", "Poke"]},
            {"name": "Hanzo", "category": ["Poke"]},
            {"name": "Hazard", "category": ["Brawl"]},  
            {"name": "Illari", "category": ["Poke"]},  
            {"name": "Queen", "category": ["Brawl"]},
            {"name": "Junk", "category": ["Brawl"]},
            {"name": "Juno", "category": ["Dive", "Brawl"]},  
            {"name": "Kiri", "category": ["Dive", "Poke", "Brawl"]}, 
            {"name": "LW", "category": ["Dive"]},
            {"name": "Lucio", "category": ["Brawl", "Dive"]}, 
            {"name": "Mauga", "category": ["Brawl"]},
            {"name": "Mei", "category": ["Brawl", "Poke"]}, 
            {"name": "Mercy", "category": ["Poke"]},
            {"name": "Moira", "category": ["Dive", "Brawl"]},
            {"name": "Orisa", "category": ["Brawl", "Poke"]},
            {"name": "Pharah", "category": ["Poke"]},
            {"name": "Ram", "category": ["Brawl", "Poke"]},
            {"name": "Reaper", "category": ["Brawl", "Dive"]},
            {"name": "Rein", "category": ["Brawl"]},
            {"name": "Hog", "category": ["Poke"]},
            {"name": "Sigma", "category": ["Poke", "Brawl"]},
            {"name": "Soj", "category": ["Poke"]},  
            {"name": "Soldier", "category": ["Poke"]},
            {"name": "Sombra", "category": ["Dive"]},
            {"name": "Sym", "category": ["Brawl"]},
            {"name": "Torb", "category": ["Poke", "Brawl"]},  
            {"name": "Tracer", "category": ["Dive"]}, 
            {"name": "Venture", "category": ["Dive", "Brawl"]},  
            {"name": "Widow", "category": ["Poke"]},  
            {"name": "Winston", "category": ["Dive"]},
            {"name": "Ball", "category": ["Dive"]}, 
            {"name": "Zarya", "category": ["Brawl"]},  
            {"name": "Zen", "category": ["Poke"]}  
            ]
    def _populate_hero_stats(self, mirrored):
        # Iterate through the rows of the data to populate hero statistics
        for idx, row in self.data.iterrows():
            result = row['Result']
            allied_heroes = row['My_Tank'].split(',') + row['My_DPS'].split(',') + row['My_Sups'].split(',') 
            enemy_heroes = row['E_Tank'].split(',') + row['E_DPS'].split(',') + row['E_Sups'].split(',')


            all_heroes = set(allied_heroes + enemy_heroes)  # Set for checking duplicates
            
            for hero in all_heroes:
                if not mirrored and hero in allied_heroes and hero in enemy_heroes:
                    continue  

                if hero not in self.hero_stats:
                    if hero in row['My_Tank'].split(',') + row['E_Tank'].split(','):
                        role = 'Tank'
                    elif hero in row['My_DPS'].split(',') + row['E_DPS'].split(','):
                        role = 'DPS'
                    elif hero in row['My_Sups'].split(',') + row['E_Sups'].split(','):
                        role = 'Support'
                    else:
                        role = 'Unknown'

                    self.hero_stats[hero] = {'matches': 0, 'wins': 0, 'Role': role}

                if hero in allied_heroes:
                    self.hero_stats[hero]['matches'] += 1
                    if result == 1:
                        self.hero_stats[hero]['wins'] += 1
                if hero in enemy_heroes:
                    self.hero_stats[hero]['matches'] += 1
                    if result == 0:
                        self.hero_stats[hero]['wins'] += 1
                
        # remove heroes with no matches
        self.hero_stats = {hero: stats for hero, stats in self.hero_stats.items() if stats['matches'] > 0}

    def get_bayesian_winrate(self, match_wins, hero_param=False, a=25, b=25):
        bayesian_winrates: dict[str, list[int]] = {}
        if hero_param:
            # Define a mapping for hero-specific (a, b) values
            hero_bayesian_params = {
            "Ana": (7, 4),  # Default Bayesian values for Ana
            "Ashe": (5, 6),
            "Bap": (1, 1),
            "Bastion": (1, 1),
            "Brig": (2, 1),
            "Cass": (0, 1),
            "DVA": (2, 1),
            "Doom": (1, 1),
            "Echo": (1, 1),
            "Genji": (2, 1),
            "Hanzo": (1, 1),
            "Hazard": (2, 1),
            "Illari": (1, 1),
            "Queen": (1, 0),
            "Junk": (1, 1),
            "Juno": (3, 4),
            "Kiri": (7, 2),
            "LW": (1, 1),
            "Lucio": (1, 1),
            "Mauga": (0, 2),
            "Mei": (4, 4),
            "Mercy": (1, 1),
            "Moira": (1, 0),
            "Orisa": (1, 1),
            "Pharah": (1, 0),
            "Ram": (2, 1),
            "Reaper": (1, 1),
            "Rein": (1, 1),
            "Hog": (1, 1),
            "Sigma": (0, 1),
            "Soj": (13, 8),
            "Soldier": (1, 1),
            "Sombra": (1, 1),
            "Sym": (0, 1),
            "Torb": (5, 3),
            "Tracer": (8, 12),
            "Venture": (1, 1),
            "Widow": (1, 1),
            "Winston": (10, 4),
            "Ball": (7, 6),
            "Zarya": (2, 2),
            "Zen": (1, 1)
        }
        for hero in match_wins:
            if hero_param:
                a, b = hero_bayesian_params[hero]
            matches = match_wins[hero]['matches']
            wins = match_wins[hero]['wins']
            bayesian_winrates[hero] = (wins + a) / (matches + a + b)  # Posterior mean of winrate
        
        return bayesian_winrates
    


    def normalise_winrates(self, winrates):
        """Normalises the winrates across roles."""
        roles = {'Tank': [], 'DPS': [], 'Support': []}
        # Separate winrates by role
        for hero, winrate in winrates.items():
            role = self.hero_stats[hero]['Role']
            roles[role].append(winrate)
        
        normalised = {}
        
        # Normalise winrates within each role
        for role, winrate_list in roles.items():
            if not winrate_list:  # Skip empty roles
                continue
            winrate_array = np.array(winrate_list)
            min_winrate = np.min(winrate_array)
            max_winrate = np.max(winrate_array)
            
            for hero, winrate in winrates.items():
                if self.hero_stats[hero]['Role'] == role:
                    normalised[hero] = (winrate - min_winrate) / (max_winrate - min_winrate)
        
        # Remove heroes with roles that were empty
        empty_roles = [role for role, winrate_list in roles.items() if not winrate_list]
        for hero in list(normalised.keys()):
            if self.hero_stats[hero]['Role'] in empty_roles:
                del normalised[hero]
        
        return normalised

    def stat_visualiser(self, values, threshold = 0.9):
        # Separate heroes by role
        roles = {'Tank': [], 'DPS': [], 'Support': []}
        for hero, value in values.items():
            role = self.hero_stats[hero]['Role']
            if value >= threshold:
                roles[role].append((hero, value))
        
        # Sort each role by value in descending order
        for role in roles:
            roles[role].sort(key=lambda x: x[1], reverse=True)
        
        # Print header
        print(f"{'Tank':<20} {'DPS':<20} {'Support':<20}")
        print("-" * 60)
        
        # Print heroes side by side
        for i in range(max(len(roles['Tank']), len(roles['Support']), len(roles['DPS']))):
            tank = f"{roles['Tank'][i][0]}: {roles['Tank'][i][1]:.4f}" if i < len(roles['Tank']) else ""
            dps = f"{roles['DPS'][i][0]}: {roles['DPS'][i][1]:.4f}" if i < len(roles['DPS']) else ""
            support = f"{roles['Support'][i][0]}: {roles['Support'][i][1]:.4f}" if i < len(roles['Support']) else ""            
            print(f"{tank:<20} {dps:<20} {support:<20}")


    def first_pick(self, teammates, curr_map, comp = "Poke,Brawl,Dive"):
        """ Calculate the total matches and wins of all heroes that were played on a given map or with specific teammates
        
        - Due to exact matching, recommended for use with priority Tank, Support, DPS for filling. 
        
        """

        comp_categories = set(comp.split(","))
        teammates = [hero.strip() for hero in teammates.split(',')] if isinstance(teammates, str) else teammates
        hero_stats: dict[str, list[int]] = {}

        for idx, row in self.data.iterrows():
            result = row['Result']
            allied_heroes = row['My_Tank'].split(',') + row['My_DPS'].split(',') + row['My_Sups'].split(',')
            enemy_heroes = row['E_Tank'].split(',') + row['E_DPS'].split(',') + row['E_Sups'].split(',')
            my_map = row['Map']

            # Check if any teammate is in allied heroes or if the map matches the current map
            self._synergy(teammates, curr_map, hero_stats, row, result, allied_heroes, enemy_heroes, my_map, comp_categories)


        for teammate in teammates:
            if teammate in hero_stats:
                del hero_stats[teammate]
        return hero_stats

    def _synergy(self, teammates, curr_map, hero_stats, row, result, allied_heroes, enemy_heroes, my_map, comp_categories):

        """ Synergy of my team with any hero pick by role. 
        - Uses exact matches of teammates.
        - Currently uses the "OR" operation between map and teammates. Will need to be refined later
        
        
        
        
        
        """
        if (all(teammate in allied_heroes for teammate in teammates) or my_map == curr_map):
            both_conditions = (all(teammate in allied_heroes for teammate in teammates) or teammates == ['']) and my_map == curr_map

            for hero in allied_heroes:
                hero_data = next((h for h in self.hero_categories if h["name"] == hero), None)
                if comp_categories.isdisjoint(hero_data['category']):
                    continue
                
                if hero not in hero_stats:
                    if hero in row['My_Tank'].split(',') + row['E_Tank'].split(','):
                        role = 'Tank'
                    elif hero in row['My_DPS'].split(',') + row['E_DPS'].split(','):
                        role = 'DPS'
                    elif hero in row['My_Sups'].split(',') + row['E_Sups'].split(','):
                        role = 'Support'
                    else:
                        role = 'Unknown'

                    hero_stats[hero] = {'matches': 0, 'wins': 0, 'Role': role}

                # Base increment
                hero_stats[hero]['matches'] += 1
                if result == 1:
                    hero_stats[hero]['wins'] += 1
                
                # Extra weighting when both conditions hold true
                if both_conditions:
                    hero_stats[hero]['matches'] += 1
                    if result == 1:
                        hero_stats[hero]['wins'] += 1


        if (all(enemy in enemy_heroes for enemy in teammates) or my_map == curr_map):
            both_conditions = (all(enemy in enemy_heroes for enemy in teammates) or teammates == ['']) and my_map == curr_map

            for hero in enemy_heroes:
                hero_data = next((h for h in self.hero_categories if h["name"] == hero), None)
                if comp_categories.isdisjoint(hero_data['category']):
                    continue

                if hero not in hero_stats:
                    if hero in row['My_Tank'].split(',') + row['E_Tank'].split(','):
                        role = 'Tank'
                    elif hero in row['My_DPS'].split(',') + row['E_DPS'].split(','):
                        role = 'DPS'
                    elif hero in row['My_Sups'].split(',') + row['E_Sups'].split(','):
                        role = 'Support'
                    else:
                        role = 'Unknown'

                    hero_stats[hero] = {'matches': 0, 'wins': 0, 'Role': role}

                # Base increment
                hero_stats[hero]['matches'] += 1
                if result == 0:
                    hero_stats[hero]['wins'] += 1

                # Extra weighting when both conditions hold true
                if both_conditions:
                    hero_stats[hero]['matches'] += 1
                    if result == 0:
                        hero_stats[hero]['wins'] += 1


    def matchup_comparison(self, a_matchup, e_matchup):
        """Input a string of heroes for both a_matchup and e_matchup."""
        a_matchup = {hero.strip() for hero in a_matchup.split(",")}
        e_matchup = {hero.strip() for hero in e_matchup.split(",")}

        results = {
            'Placeholder': {'matches': 0, 'wins': 0, 'Role': 'Unknown'}
        }

        for idx, row in self.data.iterrows():
            result = row['Result']
            allied_heroes = row['My_Tank'].split(',') + row['My_DPS'].split(',') + row['My_Sups'].split(',') 
            enemy_heroes = row['E_Tank'].split(',') + row['E_DPS'].split(',') + row['E_Sups'].split(',')

            all_heroes = set(allied_heroes + enemy_heroes)  # Set for checking duplicates

            if not((a_matchup.issubset(allied_heroes) and e_matchup.issubset(enemy_heroes)) or 
                   (a_matchup.issubset(enemy_heroes) and e_matchup.issubset(allied_heroes))):
                continue 

            results['Placeholder']['matches'] += 1   

            if (a_matchup.issubset(allied_heroes) and e_matchup.issubset(enemy_heroes) and result == 1) or \
               (a_matchup.issubset(enemy_heroes) and e_matchup.issubset(allied_heroes) and result == 0):
                results['Placeholder']['wins'] += 1

        return results


    def my_hero_performance(self):
        my_hero_perf: dict[str, list[int]] = {}
        for idx, row in self.data.iterrows():
            result = row['Result']
            my_hero = row['My_Hero']

            if my_hero in row['My_Tank'].split(','):
                my_role = 'Tank'
            elif my_hero in row['My_DPS'].split(','):
                my_role = 'DPS'
            elif my_hero in row['My_Sups'].split(','):
                my_role = 'Support'
            else:
                my_role = 'Unknown'

            if my_hero not in my_hero_perf:
                my_hero_perf[my_hero] = {'matches': 0, 'wins': 0, 'Role': my_role}

            my_hero_perf[my_hero]['matches'] += 1
            if result == 1:
                my_hero_perf[my_hero]['wins'] += 1

        return my_hero_perf
    

    def add_data(self, matches_1, matches_2):

        new_dataset = {}

        for hero_1 in matches_1:
            if hero_1 in matches_2:
                new_dataset[hero_1] = {'matches': matches_1[hero_1]['matches'] + matches_2[hero_1]['matches'], 
                                        'wins': matches_1[hero_1]['wins'] + matches_2[hero_1]['wins'], 'Role':matches_1[hero_1]['Role']}
        return new_dataset
            
            

    def matchup_hero_prob_winrate(self, a_tank, a_DPS, a_supports, e_tank, e_DPS, e_supports):
        """Calculate the win probability based on individual hero matchups for your team.
        
        
        1. Does per role comparison (meaning Tank vs Tank, Sups vs Sups, DPS vs DPS)
        2. From your team's perspective, selecting best performing heroes with and against these matchups.
        3. Currently traces over same game multiple times if filters overlap. Will need to update.
        4. Currently delete is needed but without it we don't get mirror heroes appropriately.
        
        
        
        """
        hero_winrate_prob = {}
        for idx, row in self.data.iterrows():
            result = row['Result']


            team1_dps = {hero.strip() for hero in row['My_DPS'].split(",")}
            team1_sups = {hero.strip() for hero in row['My_Sups'].split(",")}
            team1_tank = {hero.strip() for hero in row['My_Tank'].split(",")}

            team2_dps = {hero.strip() for hero in row['E_DPS'].split(",")}
            team2_sups = {hero.strip() for hero in row['E_Sups'].split(",")}
            team2_tank = {hero.strip() for hero in row['E_Tank'].split(",")}



            allied_heroes = row['My_Tank'].split(',') + row['My_DPS'].split(',') + row['My_Sups'].split(',') 
            enemy_heroes = row['E_Tank'].split(',') + row['E_DPS'].split(',') + row['E_Sups'].split(',')
            all_heroes = set(allied_heroes + enemy_heroes)  # Set for checking duplicates

           # Tank match
            if a_tank and e_tank:
                tank_match = (a_tank in team1_tank and e_tank in team2_tank) or (a_tank in team2_tank and e_tank in team1_tank)

            elif a_tank or e_tank:
                tank_match = (a_tank in team1_tank or e_tank in team2_tank) or (a_tank in team2_tank or e_tank in team1_tank)

            else:
                tank_match = False  # Neither tank is specified


            if a_DPS or e_DPS:
                # Split the provided heroes into lists
                a_DPS_heroes = [hero.strip() for hero in a_DPS.split(',')] if a_DPS else []
                e_DPS_heroes = [hero.strip() for hero in e_DPS.split(',')] if e_DPS else []

                # Check for exact match if two heroes are provided for both teams
                if len(a_DPS_heroes) == 2 or len(e_DPS_heroes) == 2:
                    dps_match = (set(a_DPS_heroes).issubset(team1_dps) and set(e_DPS_heroes).issubset(team2_dps)) or (set(a_DPS_heroes).issubset(team2_dps) and set(e_DPS_heroes).issubset(team1_dps))
                else:
                    # If only one hero is provided for a team, check if that hero is in either team
                    dps_match = (any(hero in team1_dps for hero in a_DPS_heroes) or any(hero in team2_dps for hero in a_DPS_heroes) if a_DPS_heroes else True) and \
                                (any(hero in team2_dps for hero in e_DPS_heroes) or any(hero in team1_dps for hero in e_DPS_heroes) if e_DPS_heroes else True)
                 
            else:   
                dps_match = False  # Neither DPS is specified

            
            if a_supports or e_supports:
                # Split the provided heroes into lists
                a_supports_heroes = [hero.strip() for hero in a_supports.split(',')] if a_supports else []
                e_supports_heroes = [hero.strip() for hero in e_supports.split(',')] if e_supports else []


                # Check for exact match if two supports are provided for both teams
                if len(a_supports_heroes) == 2 or len(e_supports_heroes) == 2:
                    supports_match = (set(a_supports_heroes).issubset(team1_sups) and set(e_supports_heroes).issubset(team2_sups)) or (set(a_supports_heroes).issubset(team2_sups) and set(e_supports_heroes).issubset(team1_sups))
                else:
                    # If only one hero is provided for a team, check if that hero is in either team
                    supports_match = (any(hero in team1_sups for hero in a_supports_heroes) or any(hero in team2_sups for hero in a_supports_heroes) if a_supports_heroes else True) and \
                                    (any(hero in team2_sups for hero in e_supports_heroes) or any(hero in team1_sups for hero in e_supports_heroes) if e_supports_heroes else True)
            else:   

                supports_match = False  # Neither supports are specified

          
            for hero in all_heroes:

                if hero not in hero_winrate_prob:
                    # Initialize hero data if not already done
                    if hero in row['My_Tank'].split(',') + row['E_Tank'].split(','):
                        role = 'Tank'
                    elif hero in row['My_DPS'].split(',') + row['E_DPS'].split(','):
                        role = 'DPS'
                    elif hero in row['My_Sups'].split(',') + row['E_Sups'].split(','):
                        role = 'Support'
                    else:
                        role = 'Unknown'
                    hero_winrate_prob[hero] = {'matches': 0, 'wins': 0, 'Role': role}

                if tank_match:


                    if (hero in allied_heroes and a_tank in allied_heroes) or (hero in allied_heroes and e_tank in enemy_heroes) :
                        hero_winrate_prob[hero]['matches'] += 1
                        if result == 1:
                            hero_winrate_prob[hero]['wins'] += 1

                    if (hero in enemy_heroes and a_tank in enemy_heroes) or (hero in enemy_heroes and e_tank in allied_heroes):
                        hero_winrate_prob[hero]['matches'] += 1
                        if result == 0:
                            hero_winrate_prob[hero]['wins'] += 1

                if dps_match:
                    if (hero in allied_heroes and all(dps in allied_heroes for dps in a_DPS_heroes) and bool(a_DPS_heroes)) or (hero in allied_heroes and bool(e_DPS_heroes) and all(dps in enemy_heroes for dps in e_DPS_heroes)):
                        hero_winrate_prob[hero]['matches'] += 1
                        if result == 1:
                            hero_winrate_prob[hero]['wins'] += 1
                    
                    if (hero in enemy_heroes and all(dps in enemy_heroes for dps in a_DPS_heroes) and bool(a_DPS_heroes)) or (hero in enemy_heroes and bool(e_DPS_heroes)and all(dps in allied_heroes for dps in e_DPS_heroes)):
                        hero_winrate_prob[hero]['matches'] += 1
                        if result == 0:  # Fixed: Should be 0 for enemy wins
                            hero_winrate_prob[hero]['wins'] += 1

                if supports_match:
                    if (hero in allied_heroes and all(sup in allied_heroes for sup in a_supports_heroes) and bool(a_supports_heroes)) or (hero in allied_heroes and bool(e_supports_heroes) and all(sup in enemy_heroes for sup in e_supports_heroes)):
                        hero_winrate_prob[hero]['matches'] += 1
                        if result == 1:
                            hero_winrate_prob[hero]['wins'] += 1

                    if (hero in enemy_heroes and all(sup in enemy_heroes for sup in a_supports_heroes) and bool(a_supports_heroes)) or (hero in enemy_heroes and bool(e_supports_heroes) and all(sup in allied_heroes for sup in e_supports_heroes)):
                        hero_winrate_prob[hero]['matches'] += 1
                        if result == 0:  # Fixed: Should be 0 for enemy wins
                            hero_winrate_prob[hero]['wins'] += 1




        # for hero in [a_tank] + e_tank.split(',') + a_DPS.split(',') + e_DPS.split(',') + a_supports.split(',') + e_supports.split(','):
        #     if hero in hero_winrate_prob:
        #         del hero_winrate_prob[hero]

        return hero_winrate_prob






df = pd.read_csv('stats.csv')
df = df[df['Closeness'] == 0]  # Keep only rows where closeness is 0        
heroes = best_Heroes(df)
first_pick_heroes = heroes.first_pick("Hog, Ana, Brig", "Hollywood")
bayesian_prob = heroes.get_bayesian_winrate(first_pick_heroes, True)
normalised_prob = heroes.normalise_winrates(bayesian_prob)
heroes.stat_visualiser(normalised_prob, 0.5)



Tank                 DPS                  Support             
------------------------------------------------------------
Hazard: 1.0000       Genji: 1.0000        Moira: 1.0000       
Winston: 0.9167      Sombra: 0.8000       Kiri: 0.6250        
Rein: 0.8889         Torb: 0.8000                             
DVA: 0.6667          Soj: 0.6609                              
